# NL2SQL with CSV Post-Processing

An NL2SQL agent converts natural language questions into SQL, executes them against the bookstore database, and saves results as CSV files. A second agent then operates on the CSV output using the CsvConnector. This shows how connectors chain together: SQL produces data, CSV tools refine it.

In [ ]:
from agentic_patterns.core.agents import get_agent, run_agent
from agentic_patterns.agents.nl2sql import create_agent as create_nl2sql_agent
from agentic_patterns.core.connectors.csv import CsvConnector
from agentic_patterns.core.connectors.sql.config import DBS_YAML_PATH
from agentic_patterns.core.connectors.sql.db_connection_config import (
    DbConnectionConfigs,
)
from agentic_patterns.core.connectors.sql.db_infos import DbInfos

# Bootstrap database configuration from dbs.yaml
DbConnectionConfigs.reset()
DbInfos.reset()
DbConnectionConfigs.get().load_from_yaml(DBS_YAML_PATH)

## Step 1: NL2SQL Agent Queries the Database

The NL2SQL agent receives the full bookstore schema in its instructions and has two tools: `db_execute_sql_tool` (run SQL, save CSV) and `db_get_row_by_id_tool` (fetch a single row). We ask a natural language question and the agent generates and executes SQL.

In [ ]:
nl2sql_agent = create_nl2sql_agent(db_id="bookstore")

query = "List all books with their author name and average review rating, sorted by rating descending. Save results to /workspace/books_ratings.csv"

result, nodes = await run_agent(nl2sql_agent, query, verbose=True)
print(f"\nAgent output:\n{result.result.output}")

## Step 2: CSV Agent Operates on the Results

The SQL query produced a CSV file at `/workspace/books_ratings.csv`. Now we create a second agent with CsvConnector tools to inspect and transform the data -- no SQL needed.

In [ ]:
csv_connector = CsvConnector()

csv_tools = [
    csv_connector.head,
    csv_connector.find_rows,
    csv_connector.headers,
    csv_connector.delete_rows,
    csv_connector.read_row,
]

csv_agent = get_agent(tools=csv_tools)

csv_prompt = """Using the CSV file at /workspace/books_ratings.csv:
1. Show me the column headers.
2. Show me the first 5 rows.
3. Read row 1 in detail."""

result, nodes = await run_agent(csv_agent, csv_prompt, verbose=True)
print(f"\nAgent output:\n{result.result.output}")

## Verify the CSV File on Disk

The CSV file exists on the host filesystem, persisted beyond the agent conversation.

In [ ]:
from pathlib import PurePosixPath
from agentic_patterns.core.workspace import workspace_to_host_path

host_path = workspace_to_host_path(PurePosixPath("/workspace/books_ratings.csv"))
print(f"Host path: {host_path}")
print(f"Size: {host_path.stat().st_size} bytes")
print(f"\nFirst 5 lines:\n{chr(10).join(host_path.read_text().splitlines()[:6])}")